In [2]:
## 导入库 
import numpy as np
import pandas as pd
from CoolProp import CoolProp
k = 3
a = np.array([1, 2, k])
c = np.dot(a, a.T)
print(c)


14


In [4]:
## 参数输入（量纲为质量（kg）、长度（m）、时间（s）、温度（K）、物质的量（mol））
# 可变参数
t = 8*3600                  # 系统运行的时间
Q_c = 4000000               # 物料需要的能量

# 固定参数
m_v = 6.94e-4               # 蒸汽质量流量
gamma = 2257.1*1000         # 汽化潜热
M = 18.015e-6               # 水蒸气摩尔质量
R = 8.3145                  # 气体常数
R_g = R/M                   # 水蒸气的气体常数
c_pw = 4188.2               # 水的等压比热容
T_0 = 293.15                # 环境温度
eta_steamGenerator = 0.98   # 蒸汽发生器效率

# 状态点参数。第一行为水的相态，1为液态，2为气态；第二行为绝对压力(Pa)；第三行为温度(K).
statePoint = np.array([[1, 2, 2, 1], [101325, 1002700, 198540, 101325], [293.15, 453.15, 393.15, 293.15]])






IndentationError: unindent does not match any outer indentation level (<string>, line 6)

In [7]:
def exergyLoss_water(n, m, statePoint):
    '''计算n到m过程水蒸汽（水）的㶲损失'''
    DEnthalpy = CoolProp.PropsSI('H', 'T', statePoint[2][n-1], 'P', statePoint[1][n-1], 'Water') - CoolProp.PropsSI('H', 'T', statePoint[2][m-1], 'P', statePoint[1][m-1], 'Water')
    DEntropy = CoolProp.PropsSI('S', 'T', statePoint[2][n-1], 'P', statePoint[1][n-1], 'Water') - CoolProp.PropsSI('S', 'T', statePoint[2][m-1], 'P', statePoint[1][m-1], 'Water')
    E_loss = (DEnthalpy - T_0*DEntropy)*t*m_v
    return E_loss


print(exergyLoss_water(2, 3, statePoint)/1000)
    





4620.0486336267095


In [44]:
import numpy as np
np.product = np.prod


import pandas as pd
from symfit import parameters, variables, exp, log, Fit, Model
from sympy import lambdify


import pickle
data = pd.read_csv('./process/The_MC_variation_during_the_steaming_process.csv')

data = np.array(data.iloc[:, :6].values)
print(data)

MC_raw = 21.098

j = 0
n = len(data)
t = np.zeros(n)
remove = np.array([4, 6, 8])


X0 = np.zeros(n)
X = np.zeros(n)
for i in range(0, n):
    if data[i][3] == 0:
        t[j] = data[i][0]
        X0[j] = MC_raw
        j += 1
    else:
        if data[i][0] - data[i-1][0] > 0:
            t[j] = data[i][0] - data[i-1][0]
            X0[j] = data[i-1][5]
            j += 1

j = 0
tau = np.zeros(n)
T = np.zeros(n)
for i in range(0, n-1):
    if data[i+1][0] - data[i][0] == 0:
        tau[j] = data[i][2]
        T[j] = data[i][4]
        X[j] = data[i][5]
        j += 1
        
data_t = t[:j]
data_X0 = X0[:j]
data_tau = tau[:j]
data_T = T[:j]
data_X = X[:j]


 
data_t = np.delete(data_t, remove)
data_tau = np.delete(data_tau, remove)
data_T = np.delete(data_T, remove)
data_X0 = np.delete(data_X0, remove)
data_X = np.delete(data_X, remove)
print(data_t, data_tau, data_T, data_X0, data_X)

t, tau, T, X0, X = variables('t, tau, T, X0, X')
# k, a1, b1, C1, a2, b2 = parameters('k, a1, b1, C1, a2, b2')
k, a1, b1, a2, b2 = parameters('k, a1, b1, a2, b2')
# k, a1, b1, a2 = parameters('k, a1, b1, a2')

expr = (1 - exp(-k * t)) * ((a1 * T + b1) * log(tau + 1) + (a2 * T + b2)) + X0 * exp(-k * t)
model = Model({X: expr})

# 设置参数初始值及边界
# a1=-0.2335, b1=2.4007, a2=0.3830, b2=45.7341, C1=4.3970, k=0.2576

k.value, k.min, k.max = 0.2576, 0.01, 5.0
C1.value, C1.min = 4.3970, 0.1  # 避免 log(负数)
a1.value = -0.2335
b1.value = 2.4007
a2.value = 0.3830
b2.value = 45.7341

fit = Fit(model, t=data_t, tau=data_tau, T=data_T, X0=data_X0, X=data_X)

fit_result = fit.execute()

with open("fit_result.pkl", "wb") as f:
    pickle.dump(fit_result, f)
# 加载恢复
with open("fit_result.pkl", "rb") as f:
    fit_result_1 = pickle.load(f)
# 输出结果
print(fit_result_1)
print(f"R² = {fit_result_1.r_squared:.4f}")


k = fit_result.params['k']

print(k)




[[ 8.       1.       0.       0.      75.      48.63313 17.46   ]
 [ 8.       1.       8.       1.      75.      23.522   16.01   ]
 [16.       2.       8.       1.      75.      35.576   14.9    ]
 [16.       2.      16.       2.      75.      14.844   11.86   ]
 [24.       3.      16.       2.      75.      27.05933 10.96   ]
 [24.       3.      24.       3.      75.       9.83     7.4    ]
 [ 4.       1.       0.       0.      65.      38.446   11.26   ]
 [ 4.       1.       8.       1.      65.      24.698    9.5    ]
 [ 8.       2.       8.       1.      65.      26.382    7.89   ]
 [ 8.       2.      16.       2.      65.      20.614    6.26   ]
 [12.       3.      16.       2.      65.      27.612    7.65   ]
 [12.       3.      24.       3.      65.      17.342    5.88   ]
 [16.       4.      24.       3.      65.      18.956    7.62   ]
 [16.       4.      32.       4.      65.      14.238    5.68   ]
 [20.       5.      32.       4.      65.      20.255    7.5    ]
 [20.     

In [42]:
from symfit import variables, parameters, Model, exp, log, Fit
import numpy as np
np.product = np.prod

# 定义变量和参数
t, tau, T, X0, X = variables('t, tau, T, X0, X')
k, a1, b1, C1, a2, b2 = parameters('k, a1, b1, C1, a2, b2')

# 构建模型
expr = (1 - exp(-k * t)) * ((a1 * T + b1) * log(tau + C1) + (a2 * T + b2)) + X0 * exp(-k * t)
model = Model({X: expr})

# 设置参数初始值及边界
k.value, k.min, k.max = 0.5, 0.01, 5.0
C1.value, C1.min = 1.0, 0.1  # 避免 log(负数)

# 生成模拟数据（替换为实际数据）
t_data = np.linspace(0, 10, 100)
tau_data = np.random.uniform(1, 5, 100)
T_data = np.linspace(20, 30, 100)
X0_data = np.full(100, 0.5)

print(t_data)
X_data = model(t=t_data, tau=tau_data, T=T_data, X0=X0_data, k=0.3, a1=0.2, b1=0.1, C1=1.0, a2=-0.1, b2=0.4).X
X_data += np.random.normal(0, 0.05, 100)  # 添加噪声

# 执行拟合
print(type(X_data))
fit = Fit(model, t=t_data, tau=tau_data, T=T_data, X0=X0_data, X=X_data)
result = fit.execute()

# 输出结果
print(result)
print(f"R² = {result.r_squared:.4f}")

[ 0.          0.1010101   0.2020202   0.3030303   0.4040404   0.50505051
  0.60606061  0.70707071  0.80808081  0.90909091  1.01010101  1.11111111
  1.21212121  1.31313131  1.41414141  1.51515152  1.61616162  1.71717172
  1.81818182  1.91919192  2.02020202  2.12121212  2.22222222  2.32323232
  2.42424242  2.52525253  2.62626263  2.72727273  2.82828283  2.92929293
  3.03030303  3.13131313  3.23232323  3.33333333  3.43434343  3.53535354
  3.63636364  3.73737374  3.83838384  3.93939394  4.04040404  4.14141414
  4.24242424  4.34343434  4.44444444  4.54545455  4.64646465  4.74747475
  4.84848485  4.94949495  5.05050505  5.15151515  5.25252525  5.35353535
  5.45454545  5.55555556  5.65656566  5.75757576  5.85858586  5.95959596
  6.06060606  6.16161616  6.26262626  6.36363636  6.46464646  6.56565657
  6.66666667  6.76767677  6.86868687  6.96969697  7.07070707  7.17171717
  7.27272727  7.37373737  7.47474747  7.57575758  7.67676768  7.77777778
  7.87878788  7.97979798  8.08080808  8.18181818  8

In [31]:
from CoolProp import CoolProp
delta_enthalpy = CoolProp.PropsSI('H', 'T', 453.15, 'P', 1002700,
                                      'Water') - CoolProp.PropsSI('H', 'T', 293.15, 'P',
                                                                  101325, 'Water')
print(delta_enthalpy * 0.0002778 / 1000)


0.7481755307676494


In [54]:
import numpy as np
np.product = np.prod
import pandas as pd
from symfit import parameters, variables, exp, log, Fit, Model
from process import energyAnalysis as eA
import pickle
import os


def steaming_process(t):
    """蒸制过程的参数"""
    t = t * 3600                # 系统运行的时间
    m_v = 6.78e-4               # 蒸汽质量流量   6.94e-4
    T_0 = 293.15                # 环境温度
    eta_steamGenerator = 0.90   # 蒸汽发生器效率
    state_point = np.array([[1, 2, 2, 1], [101325, 1002700, 198540, 101325], [293.15, 453.15, 453.15, 293.15]])  # 温度 293.15, 453.15, 393.15, 293.15

    exergy_loss_steamGenerator = eA.exergy_water(1, state_point, t, T_0, m_v) - eA.exergy_water(2, state_point, t, T_0, m_v)
    exergy_loss_valve = eA.exergy_water(2, state_point, t, T_0, m_v) - eA.exergy_water(3, state_point, t, T_0, m_v)
    exergy_loss_tank = eA.exergy_water(3, state_point, t, T_0, m_v) - eA.exergy_water(4, state_point, t, T_0, m_v)

    print("蒸汽发生器的㶲损为：{:.2f} kJ".format(exergy_loss_steamGenerator / 1000))
    print("蒸汽通过控制阀㶲损为：{:.2f} kJ".format(exergy_loss_valve / 1000))
    print("蒸汽通过物料罐的㶲损为：{:.2f} kJ".format(exergy_loss_tank / 1000))
    print("蒸汽发生器的能耗为：{:.2f} kW·h".format(eA.energy_of_steam_generator(t, m_v, eta_steamGenerator) * 0.0002778 / 1000))
    return 0

def steamingMCfit(MC_raw, remove, fitData = './process/The_MC_variation_during_the_steaming_process.csv'):
    """蒸制过程物料含水率的参数拟合"""
    data = pd.read_csv(fitData)
    data = np.array(data.iloc[:, :6].values)

    j = 0
    n = len(data)
    t = np.zeros(n)


    X0 = np.zeros(n)
    X = np.zeros(n)
    for i in range(0, n):
        if data[i][3] == 0:
            t[j] = data[i][0]
            X0[j] = MC_raw
            j += 1
        else:
            if data[i][0] - data[i - 1][0] > 0:
                t[j] = data[i][0] - data[i - 1][0]
                X0[j] = data[i - 1][5]
                j += 1

    j = 0
    tau = np.zeros(n)
    T = np.zeros(n)
    for i in range(0, n - 1):
        if data[i + 1][0] - data[i][0] == 0:
            tau[j] = data[i][2]
            T[j] = data[i][4]
            X[j] = data[i][5]
            j += 1

    data_t = t[:j]
    data_X0 = X0[:j]
    data_tau = tau[:j]
    data_T = T[:j]
    data_X = X[:j]

    # 去除异常数据
    if remove.size > 0:
        data_t = np.delete(data_t, remove)
        data_tau = np.delete(data_tau, remove)
        data_T = np.delete(data_T, remove)
        data_X0 = np.delete(data_X0, remove)
        data_X = np.delete(data_X, remove)
    # print(data_t, data_tau, data_T, data_X0, data_X)

    t, tau, T, X0, X = variables('t, tau, T, X0, X')
    k, a1, b1, b2 = parameters('k, a1, b1, b2')

    expr = (1 - exp(-k * t)) * ((a1 * T + b1) * log(tau + 1) + b2) + X0 * exp(-k * t)
    model = Model({X: expr})

    # 设置参数初始值及边界
    # a1=-0.2335, b1=2.4007, a2=0.3830, b2=45.7341, C1=4.3970, k=0.2576

    k.value, k.min, k.max = 0.2576, 0.01, 5.0
    a1.value = -0.2335
    b1.value = 2.4007
    b2.value = 45.7341

    fit = Fit(model, t=data_t, tau=data_tau, T=data_T, X0=data_X0, X=data_X)

    fit_result = fit.execute()
    return fit_result

def steamingMC(t, tau, T, X0, MC_raw, X_predict = 'X_t', remove = [], fitData = './process/The_MC_variation_during_the_steaming_process.csv'):
    """
    蒸制过程物料含水率的预测
    :param t: 蒸制时间
    :param tau: 干制总时间
    :param T: 干制的平均温度
    :param X0: 蒸制前的物料含水率
    :param MC_raw: 原料含水率
    :param remove: 要删除的数据的索引
    :param fitData: 蒸制过程物料含水率的数据
    :param X_predict: 预测值类型，'X_t'表示蒸制后的物料含水率，'X_eq'表示最大物料含水率
    :return: 蒸制后的物料含水率预测值
    """
    t = np.array(t)
    tau = np.array(tau)
    T = np.array(T)
    X0 = np.array(X0)
    remove = np.array(remove)

    if os.path.isfile('./process/fit_result.pkl'):
        print('物料含水率预测模型已经存在。')
    else:
        print('物料含水率预测模型拟合中...')
        fit_result = steamingMCfit(MC_raw, remove, fitData)
        print(fit_result)
        # print(f"R² = {fit_result.r_squared:.4f}")
        # 保存整个fit_result对象
        with open("process/fit_result.pkl", "wb") as f:
            pickle.dump(fit_result, f)
        # 加载恢复
    with open("process/fit_result.pkl", "rb") as f:
        fit_result = pickle.load(f)

    k = fit_result.params['k']
    a1 = fit_result.params['a1']
    b1 = fit_result.params['b1']
    b2 = fit_result.params['b2']

    t = np.array(t)
    tau = np.array(tau)
    T = np.array(T)
    X0 = np.array(X0)

    if X_predict == 'X_t':
        return (1 - np.exp(-k * t)) * ((a1 * T + b1) * np.log(tau + 1) + b2) + X0 * exp(-k * t)
    elif X_predict == 'X_eq':
        return (a1 * T + b1) * np.log(tau + 1) + b2

In [68]:
import numpy as np
np.product = np.prod
import pandas as pd
from symfit import parameters, variables, exp, log, Fit, Model
from process import energyAnalysis as eA
import pickle
import os


fitData = './process/The_MC_variation_during_the_steaming_process.csv'
MC_raw = 21.098

data = pd.read_csv(fitData)
data = np.array(data.iloc[:, :7].values)

j = 0
n = len(data)
t = np.zeros(n)

X0 = np.zeros(n)
X = np.zeros(n)
for i in range(0, n):
    if data[i][3] == 0:
        t[j] = data[i][0]
        X0[j] = MC_raw
        j += 1
    else:
        if data[i][0] - data[i - 1][0] > 0:
            t[j] = data[i][0] - data[i - 1][0]
            X0[j] = data[i - 1][5]
            j += 1

j = 0
tau = np.zeros(n)
T = np.zeros(n)
energy = np.zeros(n)
for i in range(0, n - 1):
    if data[i + 1][0] - data[i][0] == 0:
        tau[j] = data[i][2]
        T[j] = data[i][4]
        energy[j] = data[i][6]
        j += 1
        

data_t = t[:j]
data_X0 = X0[:j]
data_tau = tau[:j]
data_T = T[:j]
data_energy = energy[:j]

remove = []
remove = np.array(remove)

if remove.size > 0:
    data_t = np.delete(data_t, remove)
    data_tau = np.delete(data_tau, remove)
    data_T = np.delete(data_T, remove)
    data_X0 = np.delete(data_X0, remove)
    
data_Xeq = steamingMC(data_t, data_tau, data_T, data_X0, MC_raw, 'X_eq')

data_power = data_Xeq - data_X0
data_quantity = data_power / data_t
data_mv = data_energy * 0.9 / 0.748175531 / data_t / 3600 * 10000

print(data_power, data_quantity, data_mv)

po, qua, mv = variables('po, qua, mv')
a1, a2, a3, b = parameters('a1, a2, a3, b')

expr = (a1*po*po + a2*po + a3) * log(b*qua)
model = Model({mv: expr})

# 设置参数初始值及边界
# a1=0.0060, a2=-0.2880, a3=4.3711, b=19.4463

b.value, b.min = 1, 1
a1.value = 0.1
a2.value = 0.1
a3.value = 0.1

fit = Fit(model, po=data_power, qua=data_quantity, mv=data_mv)

fit_result_mv = fit.execute()
print(fit_result_mv)



物料含水率预测模型已经存在。
[35.3187629  12.42105719 15.17292339 35.3187629  13.09293439 11.78567041
 11.78841733 12.53893869 13.25288725 35.3187629   2.19714158  6.18841744
  8.97917849] [4.41484536 1.55263215 1.89661542 8.82969072 3.2732336  2.9464176
 2.94710433 3.13473467 3.31322181 8.82969072 0.5492854  1.54710436
 2.24479462] [ 7.29274051  6.22347271  4.57780275  9.40621513  6.59103352  6.3905458
  6.36548484  6.26524098  6.22347271 10.17475136  9.67353208  8.8966422
  8.47060581]

Parameter Value        Standard Deviation
a1        5.090152e-03 1.656226e-03
a2        -2.322435e-01 7.874124e-02
a3        3.402235e+00 9.840246e-01
b         4.747102e+01 4.791855e+01
Status message         CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Number of iterations   81
Objective              <symfit.core.objectives.LeastSquares object at 0x00000217A34AF6B0>
Minimizer              <symfit.core.minimizers.LBFGSB object at 0x00000217A34AF020>

Goodness of fit qualifiers:
chi_squared            2.892

In [ ]:
import numpy as np


k = np.zeros(1, 2) 
print(k)

In [2]:
for i in range(3):
    print(i) 

0
1
2


In [40]:
import numpy as np
np.product = np.prod
import pandas as pd
from symfit import parameters, variables, exp, log, Fit, Model
from process import energyAnalysis as eA
import pickle
import os
import random


model_type = [1, 1]

fitData = './process/The_MC_variation_during_the_steaming_process.csv'
data_all = pd.read_csv(fitData)
data_all =np.array(data_all).reshape(-1,5)
data = np.array(random.choices(data_all, k=10))
MC_0 = data[:,3]
print(MC_0)

[22.85833 22.85833 21.098   14.844   21.098   20.614   21.098   21.098
 14.844   17.342  ]


In [72]:
import numpy as np
np.product = np.prod
import pandas as pd
from symfit import parameters, variables, exp, log, Fit, Model
from process import energyAnalysis as eA
import pickle
import os
import random
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency



def modelchoiceandfit(model_type):

    fitData = './process/The_MC_variation_during_the_steaming_process.csv'
    data_all = pd.read_csv(fitData)
    data_all =np.array(data_all).reshape(-1,5)
    data = np.array(random.choices(data_all, k=10))
    
    ## 模型选择   
    match model_type[0]:
        case 1:
            # Newton             
            if model_type[1] == 1:
                t_s, MC_0, MC = variables('t_s, MC_0, MC')
                k, MC_eq = parameters('k, MC_eq') 
                expr = MC_eq - (MC_eq - MC_0) * exp(-k * t_s)
                model = Model({MC: expr})
                k.value = 1
                MC_eq.value = 0.1           
            else:
                hat_T_d, tau_d, t_s, MC_0, MC = variables('hat_T_d, tau_d, t_s, MC_0, MC')
                a1, b1, C2, k = parameters('a1, b1, C2, k')
                expr = ((a1 * hat_T_d + b1) * log(tau_d + 1) + C2) - (((a1 * hat_T_d + b1) * log(tau_d + 1) + C2) - MC_0) * exp(-k * t_s)
                model = Model({MC: expr})
                a1.value = 0.1
                b1.value = 0.1
                C2.value = 0.1
                k.value = 1
        case 2:
            # Page
            if model_type[1] == 1:
                t_s, MC_0, MC = variables('t_s, MC_0, MC')
                k, MC_eq, n = parameters('k, MC_eq, n') 
                expr = MC_eq - (MC_eq - MC_0) * exp(-k * t_s ** n)
                model = Model({MC: expr})
                k.value = 1
                MC_eq.value = 0.1
                n.value, n.min, n.max = 1, 0.1, 5
            else:
                hat_T_d, tau_d, t_s, MC_0, MC = variables('hat_T_d, tau_d, t_s, MC_0, MC')
                a1, b1, C2, k, n = parameters('a1, b1, C2, k, n')
                expr = ((a1 * hat_T_d + b1) * log(tau_d + 1) + C2) - (((a1 * hat_T_d + b1) * log(tau_d + 1) + C2) - MC_0) * exp(-k * t_s ** n)
                model = Model({MC: expr})
                a1.value = 0.1
                b1.value = 0.1
                C2.value = 0.1
                k.value = 1
                n.value, n.min, n.max = 1, 0.1, 5
        case 3:
            # Approximation of diffusion
            if model_type[1] == 1:
                t_s, MC_0, MC = variables('t_s, MC_0, MC')
                k_1, MC_eq, k_2, a = parameters('k_1, MC_eq, k_2, a') 
                expr = (MC_eq - MC_0) * (a * exp(-k_1 * t_s) + (1-a) * exp(-k_2 * t_s)) + MC_0
                model = Model({MC: expr})
                k_1.value = 1
                MC_eq.value = 0.1
                k_2.value = 1
                a.value = 0.1
            else:
                hat_T_d, tau_d, t_s, MC_0, MC = variables('hat_T_d, tau_d, t_s, MC_0, MC')
                a1, b1, C2, k_1, k_2, a = parameters('a1, b1, C2, k_1, k_2, a')
                expr = (((a1 * hat_T_d + b1) * log(tau_d + 1) + C2) - MC_0) * (a * exp(-k_1 * t_s) + (1-a) * exp(-k_2 * t_s)) + MC_0
                model = Model({MC: expr})
                a1.value = 0.1
                b1.value = 0.1
                C2.value = 0.1
                k_1.value = 1
                k_2.value = 1
                a.value = 0.1
        case 4:
            # Two-term
            if model_type[1] == 1:
                t_s, MC_0, MC = variables('t_s, MC_0, MC')
                k_1, MC_eq, k_2, a, b = parameters('k_1, MC_eq, k_2, a, b') 
                expr = (MC_eq - MC_0) * (a * exp(-k_1 * t_s) + b * exp(-k_2 * t_s)) + MC_0
                model = Model({MC: expr})
                k_1.value = 1
                MC_eq.value = 0.1
                k_2.value = 1
                a.value = 0.1
                b.value = 0.1
            else:
                hat_T_d, tau_d, t_s, MC_0, MC = variables('hat_T_d, tau_d, t_s, MC_0, MC')
                a1, b1, C2, k_1, k_2, a, b = parameters('a1, b1, C2, k_1, k_2, a, b')
                expr = (((a1 * hat_T_d + b1) * log(tau_d + 1) + C2) - MC_0) * (a * exp(-k_1 * t_s) + b * exp(-k_2 * t_s)) + MC_0
                model = Model({MC: expr})
                a1.value = 0.1
                b1.value = 0.1
                C2.value = 0.1
                k_1.value = 1
                k_2.value = 1
                a.value = 0.1
                b.value = 0.1
    
    if model_type[1] == 1:
        fit = Fit(model, MC_0=data[:,3], t_s=data[:,0], MC=data[:,4])
    else:
        fit = Fit(model, MC_0=data[:,3], t_s=data[:,0], MC=data[:,4], hat_T_d=data[:,2], tau_d=data[:,1])
    
    fit_result = fit.execute()
    print(fit_result)
    return fit_result

def modelassessment(fit_result, model_type):
    fitData = './process/The_MC_variation_during_the_steaming_process.csv'
    data_all = pd.read_csv(fitData)
    data_all =np.array(data_all).reshape(-1,5)
    t_s = data_all[:,0]
    MC_0 = data_all[:,3]
    MC = data_all[:,4]
    hat_T_d = data_all[:,2]
    tau_d = data_all[:,1]
    
    match model_type[0]:
        case 1:
            # Newton
            if model_type[1] == 1:
                k = fit_result.params['k']
                MC_eq = fit_result.params['MC_eq']
                MC_pred = MC_eq - (MC_eq - MC_0) * np.exp(-k * t_s)
            else:
                k = fit_result.params['k']
                a1 = fit_result.params['a1']
                b1 = fit_result.params['b1']
                C2 = fit_result.params['C2']
                MC_pred = ((a1 * hat_T_d + b1) * np.log(tau_d + 1) + C2) - (((a1 * hat_T_d + b1) * np.log(tau_d + 1) + C2) - MC_0) * np.exp(-k * t_s)
        case 2:
            # Page
            if model_type[1] == 1:
                k = fit_result.params['k']
                MC_eq = fit_result.params['MC_eq']
                n = fit_result.params['n']
                MC_pred = MC_eq - (MC_eq - MC_0) * np.exp(-k * t_s ** n)
            else:
                k = fit_result.params['k']
                a1 = fit_result.params['a1']
                b1 = fit_result.params['b1']
                C2 = fit_result.params['C2']
                n = fit_result.params['n']
                MC_pred = ((a1 * hat_T_d + b1) * np.log(tau_d + 1) + C2) - (((a1 * hat_T_d + b1) * np.log(tau_d + 1) + C2) - MC_0) * np.exp(-k * t_s ** n)
        case 3:
            # Approximation of diffusion
            if model_type[1] == 1:
                k_1 = fit_result.params['k_1']
                MC_eq = fit_result.params['MC_eq']
                k_2 = fit_result.params['k_2']
                a = fit_result.params['a']
                MC_pred = (MC_eq - MC_0) * (a * np.exp(-k_1 * t_s) + (1-a) * np.exp(-k_2 * t_s)) + MC_0
            else:
                k_1 = fit_result.params['k_1']
                a1 = fit_result.params['a1']
                b1 = fit_result.params['b1']
                C2 = fit_result.params['C2']
                k_2 = fit_result.params['k_2']
                a = fit_result.params['a']
                MC_pred = (((a1 * hat_T_d + b1) * np.log(tau_d + 1) + C2) - MC_0) * (a * np.exp(-k_1 * t_s) + (1-a) * np.exp(-k_2 * t_s)) + MC_0
        case 4:
            # Two-term
            if model_type[1] == 1:
                k_1 = fit_result.params['k_1']
                MC_eq = fit_result.params['MC_eq']
                k_2 = fit_result.params['k_2']
                a = fit_result.params['a']
                b = fit_result.params['b']
                MC_pred = (MC_eq - MC_0) * (a * np.exp(-k_1 * t_s) + b * np.exp(-k_2 * t_s)) + MC_0
            else:
                k_1 = fit_result.params['k_1']
                a1 = fit_result.params['a1']
                b1 = fit_result.params['b1']
                C2 = fit_result.params['C2']
                k_2 = fit_result.params['k_2']
                a = fit_result.params['a']
                b = fit_result.params['b']
                MC_pred = (((a1 * hat_T_d + b1) * np.log(tau_d + 1) + C2) - MC_0) * (a * np.exp(-k_1 * t_s) + b * np.exp(-k_2 * t_s)) + MC_0
            
    R2 = r2_score(MC, MC_pred)
    chi2, p, dof, ex = chi2_contingency(np.array([MC, MC_pred]))
    return R2, chi2, 1-p, dof






In [92]:
fit_result = modelchoiceandfit([1, 2])
print(modelassessment(fit_result, [1, 2]))



Parameter Value        Standard Deviation
C2        -7.199196e+05 1.419361e+07
a1        -1.800414e+02 4.524090e+03
b1        1.702333e+05 3.361715e+06
k         -5.116055e-06 1.008595e-04
Status message         Maximum number of iterations has been exceeded.
Number of iterations   800
Objective              <symfit.core.objectives.LeastSquares object at 0x0000021B3315B3B0>
Minimizer              <symfit.core.minimizers.BFGS object at 0x0000021B331AB800>

Goodness of fit qualifiers:
chi_squared            76.93993387183127
objective_value        38.469966935915636
r_squared              0.8863740604124092
(0.7717366694820313, np.float64(3.9131138729605257), np.float64(0.015046359201050241), 12)


In [ ]:

data_t = np.array([8, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

tau_s = np.array([])
tau_s[0] = data_t[0]
for i in range(1, len(data_t)):       
    if data_t[i] == data_t[i-1]:
        tau_s[i] = tau_s[i-1] + data_t[i]
    else:
        tau_s[i] = data_t[i]
print(tau_s)

In [17]:
import csv
import pandas as pd
import numpy as np

C = np.array([12.13376948, 6.876915825, 4.300802151, 12.93937476, 10.24416036, 8.276987612, 6.549559688, 5.219274922, 5.615255426, 23.6117448, 15.79785864, 11.68997219, 9.510836798])
C = C.reshape(-1, 1).tolist()
C.insert(0, ["coff_result"])
print(C)
with open('./process/coff_result.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(C)

[['coff_result'], [12.13376948], [6.876915825], [4.300802151], [12.93937476], [10.24416036], [8.276987612], [6.549559688], [5.219274922], [5.615255426], [23.6117448], [15.79785864], [11.68997219], [9.510836798]]


In [58]:
from CoolProp import CoolProp
import numpy as np

T_d = 65+273.15
R = 8.31446261815324
M = 18.015
Delta_m = np.array([25.21765201, 14.90857989, 10.33462179, 11.70131709, 3.893635863, 6.771376924, 2.677935944, 4.801165294, 4.976873173, 5.465968001, 8.279321715, 7.369965891, 7.661594776])

A_m = np.pi*0.8**2*10/4
t_d = 8*3600
p_s = CoolProp.PropsSI("P", "T", T_d, "Q", 1, "Water")  # 水的饱和压力 (Pa)
p_v = 1.17*1000

alpha = 2*Delta_m / (A_m*t_d*(p_s-p_v))*np.sqrt((2*np.pi*T_d*R*1000)/M)*(2-0.02)/0.04/4
print(alpha)



[1.78846565e-04 1.05733408e-04 7.32943577e-05 8.29871221e-05
 2.76141252e-05 4.80234045e-05 1.89922378e-05 3.40504310e-05
 3.52965720e-05 3.87652903e-05 5.87179269e-05 5.22686681e-05
 5.43369346e-05]


In [45]:
t_steaming = 8                  # 蒸制的时间(h)
MC_raw = 21.098                 # 原料的含水率(%)
EC_raw = 0                      # 原料的能耗(kWh/kg)
WEP_raw = 49.87472              # 原料水提多糖含量(%)
AEP_raw = 2.67019               # 原料碱提多糖含量(%)
L_raw = 27.558                  # 原料明度L*
b_raw = 8.781                   # 原料黄蓝度b*
m_raw = 50.0                    # 原料质量(kg)
T_environment = 20              # 环境温度(℃)
global raw
raw = [MC_raw, EC_raw, WEP_raw, AEP_raw, L_raw, b_raw]

print(raw[0])

21.098


In [13]:
import numpy as np
import pandas as pd
from process import energyAnalysis as eA
from CoolProp import CoolProp
from process import steamingProcess as sP
from symfit import parameters, variables, exp, log, Fit, Model
import pickle
import globals
import os
import csv

with open("process/资源包/13组干制含水率修正/fit_result_coff.pkl", "rb") as f:
    fit_result = pickle.load(f)
    print(fit_result)




Parameter Value        Standard Deviation
a3        3.695540e-02 6.971413e-03
b3        -7.925813e-01 2.685109e-01
c3        -5.628676e+00 9.286306e-01
d3        1.856327e-02 4.075607e-03
f         2.047026e+02 3.202489e+01
Status message         Desired error not necessarily achieved due to precision loss.
Number of iterations   17
Objective              <symfit.core.objectives.LeastSquares object at 0x0000022B8F713680>
Minimizer              <symfit.core.minimizers.BFGS object at 0x0000022B8F7441D0>

Goodness of fit qualifiers:
chi_squared            11.42240814054361
objective_value        5.711204070271805
r_squared              0.9757526300445981
